# Assignment Walkthrough: Electricity Consumption Prediction


## Problem Statement
An electricity utility company wants to predict the weekly electricity consumption ($y$) based on operating hours ($x$).

**Model:** Linear Regression $\hat{y} = w_0 + w_1 x$

**Loss Function:** Squared Error $J(w_0, w_1) = \frac{1}{2}\sum_{i=1}^{5}(\hat{y}_i - y_i)^2$

**Dataset:**
| Shop | Operating Hours (x) | Consumption (y) |
| --- | --- | --- |
| 1 | 1 | 3 |
| 2 | 2 | 5 |
| 3 | 4 | 9 |
| 4 | 6 | 13 |
| 5 | 8 | 17 |

In [ ]:
import numpy as np

# Dataset
X = np.array([1, 2, 4, 6, 8])
Y = np.array([3, 5, 9, 13, 17])
N = len(X)

print("Dataset X:", X)
print("Dataset Y:", Y)

Dataset X: [1 2 4 6 8]
Dataset Y: [ 3  5  9 13 17]


## Part (a): Loss Function & Convexity

### 1. Loss Function
The squared error loss function is:
$$ J(w_0, w_1) = \frac{1}{2} \sum_{i=1}^{5} (w_0 + w_1 x_i - y_i)^2 $$

### 2. Hessian Matrix
The Hessian matrix $H$ contains the second-order partial derivatives:
$$ H = \begin{bmatrix} \frac{\partial^2 J}{\partial w_0^2} & \frac{\partial^2 J}{\partial w_0 \partial w_1} \\ \frac{\partial^2 J}{\partial w_1 \partial w_0} & \frac{\partial^2 J}{\partial w_1^2} \end{bmatrix} $$

Where:
- $\frac{\partial^2 J}{\partial w_0^2} = N$
- $\frac{\partial^2 J}{\partial w_0 \partial w_1} = \sum x_i$
- $\frac{\partial^2 J}{\partial w_1^2} = \sum x_i^2$

In [ ]:
def get_hessian(X):
    h_00 = len(X)       # N
    h_01 = np.sum(X)    # Sum of x
    h_10 = np.sum(X)    # Sum of x
    h_11 = np.sum(X**2) # Sum of x^2
    return np.array([[h_00, h_01], [h_10, h_11]])

H = get_hessian(X)
print("Hessian Matrix:\n", H)

Hessian Matrix:
 [[  5  21]
 [ 21 121]]


### 3. Convexity Proof
A function is strictly convex if its Hessian is Positive Definite. We can verify this by checking if all eigenvalues are positive.

In [ ]:
eigenvalues = np.linalg.eigvals(H)
print("Eigenvalues:", eigenvalues)

if np.all(eigenvalues > 0):
    print("Conclusion: The Hessian is Positive Definite, so the Loss Function is Strictly Convex.")
else:
    print("Conclusion: The Loss Function is NOT Strictly Convex.")

Eigenvalues: [  1.31531795 124.68468205]
Conclusion: The Hessian is Positive Definite, so the Loss Function is Strictly Convex.


## Part (b): Gradient Calculation

The gradient vector $\nabla J$ is composed of the first derivatives:
$$ \nabla J = \begin{bmatrix} \frac{\partial J}{\partial w_0} \\ \frac{\partial J}{\partial w_1} \end{bmatrix} = \begin{bmatrix} \sum (\hat{y}_i - y_i) \\ \sum (\hat{y}_i - y_i)x_i \end{bmatrix} $$

In [ ]:
def get_loss(w0, w1, X, Y):
    y_pred = w0 + w1 * X
    error = y_pred - Y
    loss = 0.5 * np.sum(error**2)
    return loss

def get_gradient(w0, w1, X, Y):
    y_pred = w0 + w1 * X
    error = y_pred - Y
    grad_w0 = np.sum(error)
    grad_w1 = np.sum(error * X)
    return np.array([grad_w0, grad_w1])

w_initial = np.array([2.0, 2.0])
grad_initial = get_gradient(w_initial[0], w_initial[1], X, Y)
print(f"Initial Gradient at w={w_initial}: {grad_initial}")

Initial Gradient at w=[2. 2.]: [ 5. 21.]


## Part (c): Gradient Descent Iterations

We perform two iterations of Gradient Descent starting from $w^{(0)} = [2, 2]^T$.

**Update Rule:** $w^{(t+1)} = w^{(t)} - \eta \nabla J(w^{(t)})$

### Scenario 1: Constant Learning Rate ($\eta = 0.05$)

In [ ]:
print("--- Scenario 1: Constant LR ---")
w = w_initial.copy()
eta = 0.05

for i in range(1, 3):
    grad = get_gradient(w[0], w[1], X, Y)
    loss = get_loss(w[0], w[1], X, Y)
    print(f"Iter {i-1}: w = {w}, Loss = {loss}, Grad = {grad}")

    w = w - eta * grad

print(f"Iter 2: w = {w}, Loss = {get_loss(w[0], w[1], X, Y)}")

--- Scenario 1: Constant LR ---
Iter 0: w = [2. 2.], Loss = 2.5, Grad = [ 5. 21.]
Iter 1: w = [1.75 0.95], Loss = 51.57000000000001, Grad = [ -18.3 -111.3]
Iter 2: w = [2.665 6.515], Loss = 1398.103650000001


### Scenario 2: Decaying Learning Rate
$\eta_t = \eta_0 e^{-kt}$, where $\eta_0 = 0.1, k = 0.4$.

In [ ]:
print("--- Scenario 2: Decaying LR ---")
w = w_initial.copy()
eta_0 = 0.1
k = 0.4

for i in range(1, 3):
    # Calculate eta for the current step (t=1, then t=2)
    eta_t = eta_0 * np.exp(-k * i)

    grad = get_gradient(w[0], w[1], X, Y)
    loss = get_loss(w[0], w[1], X, Y)
    print(f"Iter {i-1}: w = {w}, Loss = {loss:.2f}, Grad = {grad}, eta_{i} = {eta_t:.5f}")

    w = w - eta_t * grad

print(f"Iter 2: w = {w}, Loss = {get_loss(w[0], w[1], X, Y):.2f}")

--- Scenario 2: Decaying LR ---
Iter 0: w = [2. 2.], Loss = 2.50, Grad = [ 5. 21.], eta_1 = 0.06703
Iter 1: w = [1.66483998 0.5923279 ], Loss = 101.33, Grad = [ -26.23691415 -156.36668418], eta_2 = 0.04493
Iter 2: w = [2.84374052 7.61833593], Loss = 2135.76


## Part (d): Stability Analysis

**Observation:** Both scenarios diverged (loss increased significantly).

**Why?** The learning rates used (0.05 and ~0.067) are too large relative to the curvature of the loss function.
The maximum stable learning rate is generally $\frac{2}{\lambda_{max}}$, where $\lambda_{max}$ is the largest eigenvalue of the Hessian.

Let's check the limit:

In [ ]:
max_stable_lr = 2 / np.max(eigenvalues)
print(f"Max Stable Learning Rate: {max_stable_lr:.5f}")

Max Stable Learning Rate: 0.01604


Since our learning rates ($0.05$ and $0.067$) are much larger than $0.016$, the optimization overshoots and diverges.

## Part (e): Line Search Methods

We search for the optimal step size $\eta$ along the steepest descent direction $d = -\nabla J(w^{(0)})$.
We minimize $\phi(\eta) = J(w^{(0)} + \eta d)$.

In [ ]:
d = -grad_initial
print(f"Search Direction d: {d}")

def phi(eta):
    w_new = w_initial + eta * d
    return get_loss(w_new[0], w_new[1], X, Y)

print(f"phi(0) = {phi(0)}")

Search Direction d: [ -5. -21.]
phi(0) = 2.5


### 1. Binary Search
We compare $\phi(m)$ and $\phi(m + \epsilon)$ to estimate the slope and reduce the interval $[a, b]$.

In [ ]:
a, b = 0.0, 1.0
epsilon = 1e-3

print(f"Initial Interval: [{a}, {b}]")

for i in range(1, 3):
    m = (a + b) / 2
    val_m = phi(m)
    val_eps = phi(m + epsilon)

    print(f"Iter {i}: m={m}, phi(m)={val_m:.2f}, phi(m+eps)={val_eps:.2f}")

    if val_m < val_eps:
        print("  -> Increasing function. Min is to the LEFT.")
        b = m
    else:
        print("  -> Decreasing function. Min is to the RIGHT.")
        a = m
    print(f"  New Interval: [{a}, {b}]")

Initial Interval: [0.0, 1.0]
Iter 1: m=0.5, phi(m)=7006.50, phi(m+eps)=7035.01
  -> Increasing function. Min is to the LEFT.
  New Interval: [0.0, 0.5]
Iter 2: m=0.25, phi(m)=1695.25, phi(m+eps)=1709.29
  -> Increasing function. Min is to the LEFT.
  New Interval: [0.0, 0.25]


### 2. Golden-Section Search
We use interior points at $1/4$ and $3/4$ of the current interval width.

In [ ]:
a, b = 0.0, 1.0
print(f"Initial Interval: [{a}, {b}]")

for i in range(1, 3):
    width = b - a
    m1 = a + 0.25 * width
    m2 = a + 0.75 * width

    val_m1 = phi(m1)
    val_m2 = phi(m2)

    print(f"Iter {i}: M1={m1}, M2={m2}")
    print(f"  phi(M1)={val_m1:.2f}, phi(M2)={val_m2:.2f}")

    if val_m1 < val_m2:
        print("  -> phi(M1) < phi(M2). Min is in [a, M2].")
        b = m2
    else:
        print("  -> phi(M1) >= phi(M2). Min is in [M1, b].")
        a = m1
    print(f"  New Interval: [{a}, {b}]")

Initial Interval: [0.0, 1.0]
Iter 1: M1=0.25, M2=0.75
  phi(M1)=1695.25, phi(M2)=15936.25
  -> phi(M1) < phi(M2). Min is in [a, M2].
  New Interval: [0.0, 0.75]
Iter 2: M1=0.1875, M2=0.5625
  phi(M1)=932.83, phi(M2)=8899.70
  -> phi(M1) < phi(M2). Min is in [a, M2].
  New Interval: [0.0, 0.5625]
